# Pinched cylinder with TDNNS formulation
In this notebook we consider a pinched cylinder, which is clamped on the left and free on the right, and a point force is acting on the top and bottom pointing inwards. Due to symmetry only one half of the cylinder is considered with symmetry boundary conditions.

The material and geometrical properties are
$$
E = 2068.5,\qquad \nu = 0.3,\qquad r=101.6,\qquad L=304.8, \qquad t=3,
$$
the maximal point force is $P_{\max}=2000$, and we consider the TDNNS method for nonlinear Naghdi shells.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import netgen.meshing as meshing

# Geometry and material parameters
thickness = 3
r = 101.6
L = 304.8
E, nu = 2068.5, 0.3
G = E / (2 * (1 + nu))
kappa = 5 / 6

# Point force
force = CF((0, 0, -2000))

mapping = lambda x, y, z: (
    r * cos(pi - x * pi),
    L * y,
    r * sin(pi - x * pi),
)
geom = meshing.SurfaceGeometry(mapping)
# nx, ny = 16, 32
mesh = Mesh(
    geom.GenerateMesh(quads=True, nx=16, ny=16, bbbpts=[[0, 0, r]], bbbnames=["force"])
)
Draw(mesh);

At the clamped boundary we need to fix the displacement by homogeneous Dirichlet boundary conditions, the shearing field, and the hybridization variable, which has the physical meaning of the rotated angle. At the symmetry boundary the displacements have to be constraint in the obvious manner and also the hybridization variable has to have homogeneous Dirichlet boundary conditions.

In [ ]:
# Set polynomial order for displacement field u, the bending moment
# tensor m, shear gamma and hybrid variable hyb have one degree less
order = 2
# Curve mesh according to displacement order
mesh.Curve(order)

# Create finite element spaces
# Clamped at left edge (bbnd ... co-dimension 2 boundary of 3D mesh)
# and symmetry boundary condition at the bottom (called "sym")
fesU = VectorH1(
    mesh,
    order=order,
    dirichletx_bbnd="top",
    dirichlety_bbnd="top",
    dirichletz_bbnd="right|top|left",
)
fesM = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
fesH = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="right|top|left")
fesB = HCurl(mesh, order=order - 1, dirichlet_bbnd="top")
fes = fesU * fesM * fesB * fesH

# Symbolic trial functions for displacement field u,
# bending moment tensor m, shear gamma, and hybrid variable hyb
u, m, gamma, hyb = fes.TrialFunction()
# Trace needed for m, gamma and hyb as we are on the surface
m, gamma, hyb = m.Trace(), gamma.Trace(), hyb.Trace()

# Regge FESpace to avoid membrane locking
Regge = HCurlCurl(mesh, order=order - 1, discontinuous=True)

# GridFunction to store the solution
gf_solution = GridFunction(fes, name="solution")

We define the unit normal vector $\hat\nu$, edge-tangential vector $\hat \tau$ and the co-normal vector $\hat\mu = \hat\nu\times \hat \tau$ at the initial configuration.

Then the projection operator onto the tangent space, deformation gradient, Cauchy-Green, and Green tensors $\boldsymbol{P}$, $\boldsymbol{F}$, $\boldsymbol{C}$, and $\boldsymbol{E}$ are introduced.

Finally, the unit normal, edge-tangential, and co-normal vectors $\nu, \tau,\mu$ on the deformed configuration are declared, which depend on the unknown displacement field $u$.

In [ ]:
# Surface unit normal, edge-tangential, and co-normal vectors on initial configuration
nv = specialcf.normal(mesh.dim)
tv = specialcf.tangential(mesh.dim)
cnv = Cross(nv, tv)

# Projection to the surface tangent space
Ptau = Id(mesh.dim) - OuterProduct(nv, nv)
# Surface deformation gradient
Ftau = Grad(u).Trace() + Ptau
# Surface Cauchy-Green tensor
Ctau = Ftau.trans * Ftau
# Surface Green-Lagrange strain tensor
Etautau = 0.5 * (Ctau - Ptau)
# surface determinant
J = Norm(Cof(Ftau) * nv)


def PseudoInverse(mat, v):
    """Pseudo Inverse of a rank (n-1) matrix
    v needs to lie in the kernel of mat
    """
    return Inv(mat.trans * mat + OuterProduct(v, v)) * mat.trans


# Surface unit normal, edge-tangential, co-normal vectors, and director on deformed configuration
nv_def = Normalize(Cof(Ftau) * nv)
tv_def = Normalize(Ftau * tv)
cnv_def = Cross(nv_def, tv_def)
director = nv_def + PseudoInverse(Ftau, nv).trans * gamma

# Surface Hessian weighted with director on deformed configuration
H_nv_def = CF((u.Operator("hesseboundary").trans * director), dims=(3, 3))

For the angle computation of the bending energy we use an averaged unit normal vector avoiding the necessity of using information of two neighbored element at once (+ a more stable formulation using the co-normal vector instead of the unit normal vector)

<center><img src="pictures/nonsmooth_av_nv_el_nv.png" width="150"> </center>

\begin{align*}
\sum_{E\in\mathcal{E}_h}\int_E(\sphericalangle(\hat{\nu}_L,\hat{\nu}_R)-\sphericalangle(\nu_L,\nu_R)\circ\phi)\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds &= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\mu\circ\phi,P^{\perp}_{\tau\circ\phi}(\{\!\{\nu^n\}\!\}))-\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,ds,
\end{align*}
where 
$$
P^{\perp}_{\tau\circ\phi}(v)= \frac{1}{\|\boldsymbol{P}^{\perp}_{\tau\circ\phi}v\|}\boldsymbol{P}^{\perp}_{\tau\circ\phi}v,\qquad \boldsymbol{P}^{\perp}_{\tau\circ\phi}= \boldsymbol{I}-\tau\circ\phi\otimes\tau\circ\phi
$$
denotes the (nonlinear) normalized projection to the plane perpendicular to the deformed edge-tangential vector $\tau$ for measuring the correct angle.

In [ ]:
# Save clamped and symmetry boundary for updating
# averaged unit normal vector during load-steps
gf_clamped_bnd = GridFunction(FacetSurface(mesh, order=0))
gf_clamped_bnd.Set(1, definedon=mesh.BBoundaries("top"))
gf_sym_bnd = GridFunction(gf_clamped_bnd.space)
gf_sym_bnd.Set(1, definedon=mesh.BBoundaries("left|right"))


# Unit normal vector on current configuration
# Used to update averaged unit normal vector during load-steps
cf_nv_cur = Normalize(Cof(Ptau + Grad(gf_solution.components[0])) * nv)

# FE space for averaged normal vectors living only on the edges of the mesh
fesVF = VectorFacetSurface(mesh, order=order - 1)
# GridFunctions to save averaged normal vectors on deformed and initial configurations
averaged_nv = GridFunction(fesVF)
averaged_nv_init = GridFunction(fesVF)

# Initialize by averaging unit normal vectors on initial configuration
# definedon=mesh.Boundaries(".*") is needed as we interpolate on the surface mesh
averaged_nv.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
averaged_nv_init.Set(nv, dual=True, definedon=mesh.Boundaries(".*"))
# Normalize averaged normal vector on initial configuration
cf_averaged_nv_init_norm = Normalize(averaged_nv_init)
# Project averaged unit normal vector being perpendicular to deformed edge-tangent vector
# to measure correct angle on deformed configuration
cf_proj_averaged_nv = Normalize(averaged_nv - (tv_def * averaged_nv) * tv_def)

Define the material and inverse material norms $\|\cdot\|_{\mathbb{M}}^2$ and $\|\cdot\|_{\mathbb{M}^{-1}}^2$ with Young modulus $\bar{E}$ and Poisson's ratio $\bar{\nu}$
\begin{align*}
\mathbb{M} \boldsymbol{E} = \frac{\bar E}{1-\bar \nu^2}\big((1-\bar \nu)\boldsymbol{E}+\bar \nu\,\mathrm{tr}(\boldsymbol{E})\boldsymbol{P}\big),\qquad\mathbb{M}^{-1} \boldsymbol{m} = \frac{1+\bar \nu}{\bar E}\big(\boldsymbol{m}-\frac{\bar \nu}{\bar\nu+1}\,\mathrm{tr}(\boldsymbol{m})\boldsymbol{P}\big).
\end{align*}

In [ ]:
# Material norm
def MaterialNorm(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * InnerProduct(mat, mat) + nu * Trace(mat) ** 2)


# Material stress
def MaterialStress(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * mat + nu * Trace(mat) * Ptau)


# Inverse of the material norm
def MaterialNormInv(mat, E, nu):
    return (1 + nu) / E * (InnerProduct(mat, mat) - nu / (nu + 1) * Trace(mat) ** 2)

Define shell energies, where the bending energy is incorporated as a saddle point problem. We set ``condense=True`` in the bilinear form to compute the Schur complement eliminating the bending moment tensor unknowns $\boldsymbol{m}$ from the global system. Thus, we obtain a minimization problem in $(u,\alpha,\hat{\gamma})$.

In [ ]:
# Bilinear form for problem
# We define the Lagrangian of the TDNNS formulation. Therefore,
# we use Variation() such that Newton knows to build the first variation
# for the residual and the second variation for the stiffness matrix. The
# stiffness matrix  will be symmetric and we use static condensation via
# condense = True to eliminate the bending moment tensor m from the global system.
# We use .Compile() to simplify (linearize) the coefficient expression tree.

bfA = BilinearForm(fes, symmetric=True, condense=True)
# Membrane energy
# Interpolate the membrane strains Etautau into the Regge space
# to avoid membrane locking
bfA += Variation(
    0.5 * thickness * MaterialNorm(Interpolate(Etautau, Regge), E, nu) * ds
).Compile()
# Bending energy
# Element terms of bending energy
bfA += Variation(
    (
        -6 / thickness**3 * MaterialNormInv(m, E, nu)
        + InnerProduct(
            H_nv_def + (J - nv * director) * Grad(nv) - Grad(gamma), 1 / J * m
        )
    )
    * ds
).Compile()
# Boundary terms of bending energy including hybridization variable
bfA += Variation(
    (
        acos(cnv_def * cf_proj_averaged_nv)
        - acos(cnv * cf_averaged_nv_init_norm)
        + hyb * cnv
        + (PseudoInverse(Ftau, nv).trans * gamma) * cnv_def
    )
    * m[cnv, cnv]
    * ds(element_boundary=True)
).Compile()
# Shear energy
bfA += Variation(0.5 * thickness * kappa * G * gamma * gamma * ds)

# Point force. Parameter par for load-stepping below.
par = Parameter(0.0)
bfA += Variation(-par * force * u * dx(definedon=mesh.BBBoundaries("force")))

In [ ]:
# Reset the solution to zero
gf_solution.vec[:] = 0
# Extract components of solution
gfu, gfm, gfgamma, _ = gf_solution.components

# Draw displacement, bending moment tensor, shear stresses, and membrane stresses
scene = Draw(gfu, mesh, "displacement", deformation=gfu)
scene2 = Draw(Norm(gfm), mesh, "bending_stress", deformation=gfu)
scene3 = Draw(thickness * kappa * G * gfgamma, mesh, "shear_stress", deformation=gfu)
gf_membrane_strain = GridFunction(Regge)
gf_membrane_strain.Set(
    0.5 * (Grad(gfu).trans * Grad(gfu) + Grad(gfu).trans * Ptau + Ptau * Grad(gfu)),
    dual=True,
    definedon=mesh.Boundaries(".*"),
)
scene4 = Draw(
    Norm(thickness * MaterialStress(gf_membrane_strain, E, nu)),
    mesh,
    "membrane_stress",
    deformation=gfu,
)

Use Newton's method for solving and increase magnitude of right-hand side by load-steps.

The unit normal vector is averaged after each load-step.

In [ ]:
# Point of interest is the point of point force
point_P = (0, 0, r)
result_P = [(0, 0, 0)]

# Use num_steps uniform load-steps
num_steps = 40

# Thread parallel
with TaskManager():
    for steps in range(num_steps):
        par.Set((steps + 1) / num_steps)
        print("Loadstep =", steps + 1, ", F/Fmax =", (steps + 1) / num_steps * 100, "%")

        # Update averaged normal vector
        # On clamped boundary it remains the unit normal vector of the initial configuration
        # On symmetry boundary the co-normal direction of the unit normal vector is constraint
        # On the rest of the boundary it is updated by the current unit normal vector
        averaged_nv.Set(
            (1 - gf_clamped_bnd - gf_sym_bnd) * cf_nv_cur
            + gf_sym_bnd * (cf_nv_cur - (cf_nv_cur * cnv) * cnv)
            + gf_clamped_bnd * nv,
            definedon=mesh.Boundaries(".*"),
            dual=True,
        )

        # Use Newton solver with residual tolerance 1e-5 and maximal 100 iterations
        # Due to hybridization techniques we can use the sparsecholesky solver for
        # solving the Schur complement (done internally)
        # For the first 16 load-steps we use a stronger damping factor 0.1 and for
        # the rest a damping factor of 0.25.
        solvers.Newton(
            bfA,
            gf_solution,
            inverse="sparsecholesky",
            dampfactor=0.1 if steps < 16 else 0.25,
            printing=False,
            maxerr=1e-5,
            maxit=100,
        )
        # Redraw solutions
        scene.Redraw()
        scene2.Redraw()
        scene3.Redraw()
        gf_membrane_strain.Set(
            0.5
            * (Grad(gfu).trans * Grad(gfu) + Grad(gfu).trans * Ptau + Ptau * Grad(gfu)),
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )
        scene4.Redraw()

        result_P.append((gfu(mesh(*point_P, BND))))

Reference solution taken from Sze, Liu, Lo, "Popular Benchmark Problems for Geometric Nonlinear Analysis of Shells", Finite Elements in Analysis and Design, 40(11), 1551-1569, 2004.

In [ ]:
ref_Pz = [
    (0, 0.0),
    (5.421, 0.05),
    (16.100, 0.1),
    (22.195, 0.125),
    (27.657, 0.15),
    (32.700, 0.175),
    (37.582, 0.2),
    (42.633, 0.225),
    (48.537, 0.25),
    (56.355, 0.275),
    (66.410, 0.3),
    (79.810, 0.325),
    (94.669, 0.35),
    (113.704, 0.4),
    (124.751, 0.45),
    (132.653, 0.5),
    (138.920, 0.55),
    (144.185, 0.6),
    (148.770, 0.65),
    (152.863, 0.7),
    (156.584, 0.75),
    (160.015, 0.8),
    (163.211, 0.85),
    (166.200, 0.9),
    (168.973, 0.95),
    (171.505, 1.0),
]

Plot the solution and compare with reference values.

In [ ]:
import matplotlib.pyplot as plt

_, _, P_z = zip(*result_P)
y_axis = [i / num_steps for i in range(len(P_z))]
P_z = [-val for val in P_z]

P_z_ex, y_axis_ref = zip(*ref_Pz)


plt.plot(P_z, y_axis, "-*", label="$P_z$")
plt.plot(P_z_ex, y_axis_ref, "--", color="k", label="Sze et al. 2004")

plt.xlabel("displacement")
plt.ylabel("$P/P_{\\max}$")
plt.legend()
plt.show()

In [ ]:
for i, (az, y) in enumerate(zip(P_z, y_axis)):
    print(f"({az}, {y} )")